# 第五章 智能体架构概览
## 5.1 简介
在基础篇中，我们已经介绍了大语言模型（LLM）必要的基础知识，包括生成式大语言模型，提示工程，智能体评估与调试。然而，要构建一个具备自主性、持续性和复杂任务处理能力的智能体，仅依赖LLM的纯文本推理是不够的。智能体需要像人类一样具备**记忆存储**、**工具调用**、**动态规划**的能力，从而解决复杂的现实问题。

本章将探讨智能体架构的核心组件与工作流，重点分析如何通过模块化设计克服单LLM的局限性。例如，LLM的上下文窗口有限，需通过记忆模块扩展其知识存储（详情参见第6章）；LLM无法直接访问外部数据或执行操作，需通过工具接口实现（详情参见第7章）；而复杂任务的拆解则需要推理与规划模块的介入（详情参见第8章）。这些模块共同构成了智能体的“大脑”与“手脚”，使其从“被动回答问题”升级为“主动完成任务”。

### 本章重点内容
我们将在本章重点介绍以下内容：
1. 智能体核心组件
2. 智能体工作流
3. 常见智能体架构模式
4. 框架与库简介
5. 最简智能体实现


## 5.2 大模型智能体架构引言
Lilian Weng在其经典博文“[LLM Powered Autonomous Agents](https://lilianweng.github.io/posts/2023-06-23-agent/)”中提出，一个完整的智能体架构需要通过模块化设计弥补LLM的三大局限：
1. 记忆局限性：上下文窗口有限，缺乏持久化的记忆机制，训练数据存在时间边界，无法记住历史交互信息。
2. 工具使用局限性：无法访问实时数据和外部系统，难以执行复杂数学运算和数据分析，仅能处理文本，缺乏多媒体能力。
3. 推理规划局限性：缺乏多轮迭代和自我纠错，难以制定和执行多步骤计划，容易偏离原始任务目标。

正如图5.1所示，一个由LLM驱动的自主Agent架构，其中包含记忆(Memory)、工具（Tools）、规划（Planning） 三大核心组件。其中Agent处于中心位置，通过协同各种组件来处理复杂的任务和决策过程。

- **记忆（Memory）**：用于存储长期知识和历史对话，支持上下文扩展和信息检索。通过记忆模块，智能体可以在多轮对话中保持一致性，并在需要时回溯历史信息。
- **工具（Tools）**：提供外部数据访问和操作执行能力。智能体可以调用API、数据库或其他服务来获取实时信息或执行特定任务，如查询天气、预订机票等。
- **规划（Planning）**：负责任务拆解和决策制定。智能体通过规划模块将复杂任务分解为可执行的子任务，并制定执行策略。这一过程类似于人类在面对复杂问题时的思考方式。
- **执行（Action）**：实际执行规划好的操作或调用工具，这包括与外部世界互动，或者通过工具来完成一个动作。


<div align="center">
  <img src="./assets/framework_cp5.png" alt="Agent Framework" width="100%" />
</div>



## 5.3 智能体核心组件

### 5.3.1 记忆模块（Memory）

智能体的记忆模块是其持续学习与任务执行的基础，其设计灵感部分来源于人类认知科学中的记忆分类体系（见表 5.1）。通过模拟生物记忆的层级结构，智能体能够实现对信息的动态存储与高效调用。

| **人类记忆类型**        | **智能体映射**                                                                 |
|-------------------------|-----------------------------------------------------------------------------|
| **感觉记忆**（Sensory Memory） | 对原始输入（文本、图像等）进行实时嵌入编码，作为任务执行的“感知快照”（如将句子分解为tokens,并进一步编码为词嵌入向量） |
| **短期记忆**（Short-Term Memory） | 依赖LLM的上下文窗口进行有限存储（如对话历史或任务中间结果），容量受Transformer架构限制       |
| **长期记忆**（Long-Term Memory） | 通过外部存储（如向量数据库）实现无限扩展，支持跨任务、跨时间的知识持久化与检索              |


#### **1. 感觉记忆：感知输入的即时编码**  
在人类认知中，感觉记忆（如视觉的“图像记忆”、听觉的“回声记忆”）负责将外界刺激转化为可处理的神经信号。智能体的感觉记忆模块对应**原始输入的嵌入表示**：  
- **文本输入**：通过LLM的Tokenizer将用户指令编码为向量（如GPT-4的token序列）；  
- **多模态输入**：结合CLIP[1]等模型将图像、音频等非文本信息转化为可检索的向量空间；  
- **实时感知**：在物理环境中（如机器人），通过传感器将环境状态（温度、位置）编码为结构化数据。  


#### **2. 短期记忆：上下文窗口内的动态处理**  
人类短期记忆的容量约为7±2个信息单元[2]，且持续时间有限（20-30秒）。智能体的短期记忆直接依赖LLM的上下文窗口，表现为：  
- **任务上下文缓存**：将当前对话或任务的中间结果（如工具调用参数、推理步骤）保留在提示词中；  
- **上下文学习**（In-Context Learning）：通过提供示例或思维链（Chain-of-Thought）模板，引导LLM在单次交互中临时“学习”新规则（如数学运算步骤）；  


#### **3. 长期记忆：外部存储与高效检索**  
人类长期记忆分为**显性记忆**（Explicit Memory，如事件和事实）和**隐性记忆**（Implicit Memory，如技能与习惯）。智能体的长期记忆模块通过外部存储系统实现类似功能。首先长期记忆是智能体跨会话累积的知识和信息，由于无法完整放入LLM上下文，需要通过独立的存储机制管理，通常使用**向量数据库**存储。其内容包括用户偏好资料、历史对话摘要、先前任务结果和领域知识等。长期记忆使智能体具备**持续成长**能力，能够跨会话保持一致性和连贯性，避免像"金鱼"一样每次都从零开始，从而提供个性化服务并减少重复犯错。

由于长期记忆存储在模型上下文之外，智能体需要**检索机制**来按需获取相关信息。当用户请求到来时，系统根据请求内容从长期记忆中检索匹配信息，补充到当前对话上下文中供LLM生成答案。检索方式从简单的**关键词匹配**、**规则查找**到复杂的**语义搜索**、**向量检索**等，核心目标是在海量长期记忆中精准找出与当前对话最相关的部分，弥补短期记忆的不足。


#### 智能体如何利用记忆模块

下图展示了智能体利用记忆模块来回答用户查询的基本流程。当用户发起查询后，智能体首先会在自己的大模型内部检索近期的对话内容，也就是所谓的短时记忆。由于大模型的上下文窗口有限，智能体仅能记住最近的对话内容，这就限制了它直接利用长期历史数据的能力。

为了弥补这一局限，智能体会启用检索增强生成（Retrieval-Augmented Generation, RAG）机制。RAG系统能够根据当前的查询和上下文，从外部数据库（长期记忆）中检索出与问题高度相关的信息。智能体将检索到的关键信息注入到当前的大模型上下文中，用于丰富对查询的理解和响应。最终，大模型会结合短期记忆和检索到的长期记忆内容，生成一条更为准确和全面的答复，并返回给用户。

通过这种流程，智能体能够突破只依赖短时记忆的限制，实现更加灵活、知识丰富的对话和问答体验。

<div align="center">
  <img src="./assets/Memory_cp5.png" alt="Memory Module" width="100%" />
</div>



> **扩展阅读**：第6章《记忆管理与检索增强》将深入探讨记忆模块的技术实现，包括内存管理策略、检索增强生成 (RAG)方法等，并将完成一个支持“记忆+检索”的问答助手。  


### 5.3.2 工具调用（Tool Calling）

工具接口是智能体架构中实现外部能力扩展的核心组件，其本质是通过标准化协议连接语言模型与外部工具系统（如API服务、数据库、计算引擎等）。该模块负责将智能体的决策转化为具体的工具调用指令，并解析工具返回的结果以支持后续推理。

#### 常见的工具类型
1. 信息检索类工具
如网页搜索引擎、数据库查询接口（Bing Search API，SQL API）。智能体遇到知识性问题时可使用此类工具获取外部信息。
2. 计算与执行类工具
如计算器、代码执行器（mathjs，Code Interpreter）。当需要数学运算或运行代码时，智能体可调用计算引擎或编程环境完成任务。
3. 设备控制类工具
如调用操作系统命令、发送邮件、控制物联网设备（Shell API， Gmail API）等，使智能体能对现实世界产生影响。

#### 工具接口设计（Tool Interface Design）
**工具接口设计**是把一个“函数/API/服务”以模型可理解的方式封装起来，从而使模型能够看懂，并学会如何调用工具。工具接口的有效性很大程度上依赖于工具描述的标准化程度。OpenAI 在 [GPT Function Calling](https://platform.openai.com/docs/guides/function-calling?api-mode=responses#defining-functions) 中规定了 JSON Schema 的形式，对于需要更强可控性或多参数传递的复杂工具，我们可以采用这种结构化的接口定义方式。大多数主流LLM平台(如OpenAI，阿里云通义千问等)都采用或兼容基于JSON Schema的格式来定义工具。JSON Schema 标准的一个典型示例如下：

```json
{
  "name": "get_weather",
  "description": "获取指定城市的天气信息",
  "parameters": {
    "type": "object",
    "properties": {
      "location": {"type": "string", "description": "城市名称"},
      "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]}
    },
    "required": ["location"]
  }
}

```

#### 工具调用的技术演进
工具调用能力是智能体实现外部知识交互和任务扩展的核心机制，其技术发展经历了从静态API适配到动态插件集成的演进过程。当前主流方案可分为API微调范式和实时插件系统两大类。


1. **API微调范式**
早期的工具调用技术主要通过微调语言模型来实现API的内嵌式使用。这种方法要求在模型训练阶段显式注入工具使用逻辑，典型代表包括：  
   - **TALM (Tool Augmented Language Models)**[3]：其核心创新在于构建多任务训练框架，将API调用作为语言模型的生成目标之一。通过在预训练阶段混入工具调用样本（如包含[Wikipedia API: query="current year"]的标注数据），模型能够学习到特定语境下的工具激活模式。
   - **Toolformer**[4]：采用更轻量级的训练策略，仅通过少量标注的API调用样本（如维基百科、QWEN、计算工具等）引导模型学习工具使用逻辑。
2. **实时插件系统** 
随着智能体应用场景的复杂化，模块化插件系统逐渐成为主流解决方案：
   - **ChatGPT Plugins**：作为首个大规模商业化的工具接口方案，其采用插件市场的模式允许开发者提交工具（如Expedia、Wolfram Alpha）。用户通过自然语言指令触发工具调用。例如："帮我预订从北京到纽约的最便宜航班"会激活Expedia插件，再利用参数提取模块生成{"origin": "北京", "destination": "纽约", "date": "2024-03-15"}的调用参数。
   - **OpenAI Function Calling**：该方案在2023年开发者大会上推出，其突破在于定义了标准化的JSON Schema工具描述语言。

#### 工具调用的关键挑战
1. **调用效率瓶颈**：多工具串联调用可能引发延迟累积
2. **错误处理机制**：需建立工具调用失败时的回退策略（如超时重试、替代方案生成）
3. **多轮工具调用**：复杂任务可能需要多次调用不同工具，模型需具备上下文跟踪与状态管理能力
4. **安全与权限**：涉及敏感数据的工具（如银行支付接口）需要严格的访问控制与审计追踪


#### 智能体如何利用工具模块
下图展示了智能体（Agent）如何通过调用外部工具来回答用户问题的流程。

当用户发起查询后，智能体内部的大模型首先会分析用户需求。一旦判断需要借助特定工具（比如需要计算、代码执行或网页搜索），模型会进入工具调用流程。在工具调用环节，大模型会从工具集中选择最合适的工具，如计算器、代码运行器或网络检索工具，并自动生成调用指令。这些指令被发送到目标工具后，工具将任务结果返回给大模型。大模型会处理工具结果并判断是否仍需进一步调用工具，并不断循环。最终，大模型会综合原始上下文和工具返回结果，生成针对用户问题的最终答复，并返回给用户。

<div align="center">
  <img src="./assets/Tool_cp5.png" alt="Tool Module" width="100%" />
</div>

> **扩展阅读**：在第7章《外部工具调用》中我们将更细致地学习常见的工具类型、如何选择并使用工具、多工具的整合与调度，最终将实现一个包含网络查询和计算能力的问答智能体。


### 5.3.3 复杂推理与规划模块

智能体在处理复杂任务时，需通过系统化的决策机制实现目标导向的行动序列生成。该模块的核心功能包括：**任务分解**（将宏观目标拆解为子任务）、**路径规划**（确定子任务执行顺序）和 **动态修正**（根据反馈调整策略）等。

<div align="center">
  <img src="./assets/Reasoning_cp5.png" alt="Tool Module" width="100%" />
</div>

上图展示了大语言模型在复杂任务中逐步增强推理能力的路径，涵盖从直接作答，到思维链构造，再到规划策略与搜索增强四个层级
1. **直接作答**范式：模型面对问题立即给出答案，代表基础但脆弱的能力，对多步推理任务往往无效。
2. **思维链（Chain-of-Thought, CoT）**[6]范式，通过提示词工程引导模型显式输出推理路径，例如在数学题中生成"首先计算面积，然后求体积"等逻辑链。其优势在于可解释性以及通用性，但局限性包括易产生错误积累，推理链结构过于刚性等。
3. **规划策略范式**，包含计划-执行、子任务分解与动态反思等机制，强调任务解决步骤的整体规划与执行控制。该范式下的代表应用包括**ReAct框架** [9]，**Least‑to‑Most Prompting** [12]，**Reflexion机制** [10]等方法。
4. **基于搜索的推理增强方法**，通过**思维树（Tree-of-Thoughts, ToT）**[7]、自我一致性及搜索算法结合，实现多路径探索与更强的解题鲁棒性。

> **扩展阅读**：在第8章《推理、规划和树搜索增强》中我们将更详细地了解智能体推理与规划中面临的挑战、智能体任务规划的策略以及基于搜索的推理增强，最终我们将构建可解决复杂任务的智能体。

## 5.4 智能体工作流（Agentic Workflow）
当前，绝大部分用户在运用AI大语言模型时，仍然采用最基础的交互模式——输入指令后直接获取回复。吴恩达教授将这种操作方式幽默地比作zero-shot(零样本)的“非智能体工作流”方式：我们指导大模型的方法，如同让某人坐在打字机前开始疯狂敲击键盘，甚至来不及按删除键。
与此形成鲜明对比的，是另一种更为精细的操作策略：首先让大模型产出初步结果，随后对该结果进行评估并提出优化方案，接着基于反馈重新生成改进版本，然后继续评估......在此过程中可能需要调动各类工具和不同的应用。通过这样反复进行的"分析-优化"循环，模型的推理表现将获得显著提升。这正是典型的智能体工作流（Agentic Workflow）模式。

实际上，智能体工作流可以让不那么强悍的大模型，达到更高的水平。一方面是智能体工作流的迭代循环和自我修正可以不断优化结果。另一方面，在规划阶段通过思维链(Chain-of-Thought)等方式将任务逐步分解也可以提升推理水平。**智能体的工作流融合了记忆模块、决策引擎、工具接口等组件的协同工作，其本质是构建了一个动态适应的闭环系统。**


#### 工作流核心流程  
<div align="center">
  <img src="./assets/workflow_cp5.png" alt="Agentic Workflow" width="100%" />
</div>

上图详细展示了简易的智能体工作流的关键步骤：
1. **感知输入**  
   - 接收用户指令（如"撰写商业计划书"）或环境信号（如机器人传感器数据）  
   - 解析输入语义并提取关键参数（目标、约束条件、上下文）  

2. **记忆检索**  
   - **长期记忆**：从向量数据库检索历史经验（如"上月市场分析报告"）  
   - **短期记忆**：加载当前会话的上下文（如前序对话记录、已执行步骤）  

3. **任务规划**  
   - 判断是否需要结合工具接口，调用工具 
   - 通过子目标分解，搜索增强等手段来生成候选行动方案 

4. **工具调用**  
   - 若需外部数据/计算，通过工具接口模块生成API请求
   - 处理工具返回结果并更新记忆库  

5. **循环迭代**  
   - 根据执行结果判断是否达成目标  
   - 若未完成，进入下一轮推理（可能调整子任务顺序或切换工具）；若出现错误，触发反思机制（如Reflexion模块回溯修正）  



## 5.5 常见智能体架构模式 
本节系统梳理当前主流的智能体架构范式，通过展示其技术特性、适用场景及局限性，帮助读者建立具体认知。主要模式包括基于提示的混合决策架构、计划-执行-反馈架构以及完全自主的循环架构。

### 5.5.1 基于提示的混合决策架构（Prompt-Based Hybrid Architecture）
基于提示的混合决策架构代表了智能体设计中的一种重要范式，其核心思想是通过精心设计的提示词模板将推理和行动能力统一在大语言模型的输出中。ReAct[9]作为该架构的代表框架，通过结构化的提示词设计实现了推理（Reasoning）和行动（Acting）的有机结合。

**工作原理**：  
1. 通过结构化提示词将推理（Reason）和行动（Act）统一在模型输出中  
2. 每个决策周期包含：  
   - `推理（Thought）`: 分析当前状态并生成推理路径  
   - `动作（Action）`: 调用工具或执行动作  
   - `观测（Observation）`: 接收工具反馈并更新记忆

下图展示了ReAct框架的结构和典型示例
![ReAct](./assets/ReAct_cp5.png)

**架构局限性**

尽管ReAct架构具有良好的可解释性，但其性能很大程度上依赖于提示词的设计质量。不当的提示词设计可能导致推理链条断裂或行动选择错误。此外，严格的三元循环模式在处理某些复杂任务时可能显得过于僵化，难以适应需要并行处理或非线性推理的场景。

### 5.5.2 计划-执行-反馈架构（Plan-Execute-Feedback Architecture）
计划-执行-反馈架构体现了智能体设计中的分阶段处理思想，通过将复杂任务分解为规划、执行和反馈三个相对独立的阶段来提高系统的整体性能和可靠性。**HuggingGPT** [5]框架将这一理念应用到AI模型的协调和集成中，创建了一个能够统一调度多种AI工具和模型的智能体系统。

**工作原理**：  
1. **规划阶段**：将用户请求分解为模型选择（Plan）→工具调用（Execute）→结果整合（Feedback）
2. **执行阶段**：  
   - 按顺序调用工具（如先调用数据库，再运行分析模型）  
   - 每步执行后记录状态（成功/失败，返回数据）  
3. **反馈阶段**：  
   - 根据执行结果更新计划（如跳过已验证的子任务）  
   - 通过反思机制优化后续步骤


下图展示了HuggingGPT框架的结构和典型示例 
![ReAct](./assets/HuggingGPT.png) 

**架构局限性**

计划-执行-反馈架构虽然提供了良好的模块化和可扩展性，但也面临一些挑战。首先是规划复杂性问题，对于高度复杂的任务，生成最优的执行计划本身就是一个困难的问题，可能需要大量的计算资源和时间。其次是错误传播风险，由于各个阶段之间存在依赖关系，早期阶段的错误可能会在后续执行中被放大，影响整体任务的成功率。

### 5.5.3 自主循环智能体架构（Autonomous Looping Architecture）  

自主循环架构代表了智能体设计的另一种重要范式，其核心理念是构建能够持续自主运行的智能体系统。与前述架构需要明确的任务分解或用户指导不同，自主循环架构旨在创建能够独立设定目标、制定计划并持续执行的智能体。

**代表框架**：AutoGPT [12]
**工作原理**：  
1. 接收需求：AutoGPT接收一个高层次的目标描述，工作流程为一个持续的自主循环。
2. 循环执行： 比较执行结果与目标差距，动态生成新子任务；根据执行结果动态调整优先级和执行顺序； 
3. 终止机制： 根据用户设定的硬性条件或者自动检测到目标完成


**架构局限性**

自主循环架构虽然提供了强大的自主性，但也带来了显著的挑战。首先是目标漂移问题，在长期运行过程中，智能体可能逐渐偏离原始目标，特别是在面对复杂或模糊目标时。其次是资源消耗问题，持续的自主循环运行会消耗大量的计算资源和API调用，成本控制成为重要考虑因素。此外，缺乏有效的终止条件判断机制，智能体可能陷入无效的循环执行，或者在达到目标后无法及时停止。最后，由于高度自主性，系统的可解释性和可控性相对较弱，用户难以预测和控制智能体的具体行为路径。


### 5.5.4 架构模式对比与适用性  
| 架构类型 | 任务复杂度 | 环境动态性 | 资源需求 | 典型场景 |  
|---|---|---|---|---|  
| 基于提示的混合架构 | 低-中 | 高（需实时交互） | 低 | 多跳问答、事实验证 |  
| 计划-执行-反馈架构 | 中-高 | 中（支持有限调整） | 中 | 多工具协作、结构化任务 |  
| 自主循环架构 | 高 | 低（依赖预设规则） | 高 | 长期自主任务 |  

## 5.6 框架与库简介

智能体开发框架的出现大幅降低了技术实现门槛，使开发者能够专注于业务逻辑而非底层代码。本节将介绍当前主流工具如何通过模块化设计实现智能体核心架构组件，并分析其适用场景。

### 5.6.1 LangChain：模块化开发的标杆

LangChain是一个开源的用于开发由语言模型驱动的智能体的框架，它使得智能体具有上下文感知能力和推理能力。核心定位是面向LLM应用的通用开发框架，支持Python和JavaScript双语言生态。

LangChain框架主要由以下几个部分组成：
1. LangChain 库：Python 和 JavaScript 库，用于处理语言模型的可组合工具和集成。无论你是否使用 LangChain 框架的其余部分，组件都是模块化的，易于使用。
2. LangChain模版：一系列易于部署的参考模版架构，可用于各种任务。
3. LangServe：用于将 LangChain 链部署为 REST API 的库。
4. LangSimth：开发者平台，让你可以调试、测试、评估和监控基于任何 LLM 框架构建的链，并且与 LangChain 无缝集成。


**架构组件实现**：
- **记忆模块**：提供短期记忆（ConversationBufferMemory）和长期记忆（向量数据库集成）双模式，支持Redis、Pinecone、Chroma等多种存储方案
- **工具调用**：提供丰富的预构建工具集（搜索引擎、API调用、数据库连接器等），通过统一的`Tool`接口实现标准化工具集成
- **规划引擎**：内置多种Agent类型，包括ReAct Agent、Plan-and-Execute Agent、OpenAI Functions Agent等

**开发流程优势**：
LangChain模块化设计一起简化了Agent开发的流程：
1. **开发**：在 LangChain/LangChain.js 中编写你的应用程序。使用模板作为参考，快速开始。
2. **生产化**：使用 LangSmith 来检查、测试和监控你的链。
3. **部署**：使用 LangServe 将任何链转换为 API。


### 5.6.2 LlamaIndex：非结构化数据处理专家

**核心定位**：LlamaIndex 是一个将大语言模型和外部数据连接在一起的框架，专门针对私有数据构建**智能检索**和**问答系统**的框架，强调数据连接和上下文增强。

**架构组件实现**：
- **数据连接器**：提供100+数据源连接器，涵盖文档（PDF、Word、Notion）、数据库（SQL、NoSQL）、API（Slack、GitHub）等
- **索引构建**：支持多种索引结构
  - **向量索引**：基于embedding的语义检索
  - **树形索引**：层次化文档组织
  - **关键词索引**：基于TF-IDF的精确匹配
  - **图索引**：知识图谱结构化表示
- **查询引擎**：
  - **子问题查询**：将复杂问题分解为子问题
  - **多文档查询**：跨文档信息聚合
  - **SQL查询**：结构化数据的自然语言查询
- **记忆管理**：ChatMemoryBuffer实现对话历史管理，支持向量存储和元数据过滤


### 5.6.3 Haystack：企业级信息检索框架

**核心定位**：由Deepset开源的端到端问答系统框架，专注于构建生产级的搜索和问答应用

**架构组件实现**：
- **知识库管理**：内置文档预处理Pipeline（分割、清洗、转换），支持20+文档格式，集成Sentence Transformers进行向量化
- **检索增强**：实现密集检索（Dense Passage Retrieval）、稀疏检索（BM25）和混合检索策略，支持Elasticsearch、OpenSearch、Pinecone等后端
- **决策流程**：通过Pipeline编排节点（Retriever、Reader、Generator），支持条件分支、循环逻辑和错误处理
- **评估体系**：提供检索质量评估（Recall@k、MRR）和生成质量评估（BLEU、ROUGE）等指标

**技术特点**：
- 采用声明式YAML配置管理复杂Pipeline
- 支持REST API部署和Kubernetes扩展
- 提供Haystack Studio可视化界面进行Pipeline设计和调试
- 内置A/B测试和性能监控功能

### 5.6.4 主流框架综合对比

| 框架名称 | 核心优势 | 典型应用场景 | 记忆管理 | 工具扩展性 | 部署复杂度 |
|---------|---------|-------------|---------|-----------|----------|
| **LangChain** | 通用性强，生态丰富 | 聊天机器人、RAG应用、工作流自动化 | 多种内存类型支持 | 极高（丰富工具库） | 中等 |
| **LlamaIndex** | 数据连接和检索专业 | 企业知识库、文档问答、数据分析 | 向量存储+元数据 | 高（数据连接器） | 中等 |
| **Haystack** | 企业级部署能力 | 搜索引擎、问答系统、信息抽取 | Pipeline状态管理 | 高（节点化扩展） | 高 |
| **Transformers** | 丰富的预训练模型生态 | NLP模型微调和推理 | 模型内置上下文 | 中等（需适配器开发） | 低 |
| **AutoGPT** | 自主任务执行和目标驱动 | 自动化工作流、内容生成 | JSON文件存储 | 高（插件生态） | 中等 |
| **AgentGPT** | Web界面的自主智能体平台 | 在线AI助手、任务自动化 | 会话级内存 | 中等（API集成） | 低（SaaS） |
| **Semantic Kernel** | 微软技术栈深度集成 | 企业级AI应用开发 | Azure认知服务 | 高（.NET/Python） | 中等 |
| **CrewAI** | 多智能体协作框架 | 团队协作任务、复杂项目管理 | 分布式状态管理 | 高（角色定制） | 中等 |
| **AutoGen** | 多智能体对话和协作 | 代码生成、复杂推理任务 | 对话历史管理 | 高（自定义代理） | 中等 |

### 5.6.5 框架选择建议

**新手入门**：推荐LangChain或LlamaIndex，文档完善，社区活跃

**企业应用**：选择Haystack或Semantic Kernel，提供完整的生产级解决方案

**研究原型**：使用Transformers直接构建，灵活性最高

## 5.7 实践案例：最简智能体架构实现
本节将通过一个简化的代码实例，基于LangChain框架快速构建一个具备基本智能体能力的"小助手Agent"。我们将重点使用LangChain的内置组件，让初学者能够快速理解和上手。

**Step-1. 环境准备与基础配置**

In [ ]:
# 安装必要依赖
!pip install langchain openai python-dotenv langchain-experimental numexpr duckduckgo-search google-search-results

# 导入基础库
import os
from dotenv import load_dotenv

print("✅ 环境配置完成")

**Step-2. 初始化LLM模型**

In [3]:
# -*- coding: utf-8 -*-
# 1. LLM初始化模块
from langchain.chat_models import ChatOpenAI

# 使用Deepseek API
llm = ChatOpenAI(
    model_name="deepseek-chat",
    openai_api_base="https://api.deepseek.com/v1",
    openai_api_key="sk-2ad88b35e31c44a99830ffc9e1c1465b",  # 导入自己的deepseek API密钥
    temperature=0.7,
    max_tokens=500
)

# 测试LLM是否正常工作
test_response = llm.predict("你好，请简单介绍自己")
print("🤖 Deepseek LLM测试:")
print(test_response)

🤖 Deepseek LLM测试:
你好！我是 **DeepSeek Chat**，由深度求索（DeepSeek）公司开发的智能AI助手。我的核心能力包括：  

🔹 **知识丰富**：掌握广泛的知识，涵盖科技、历史、文学、编程等多个领域，能解答各种问题。  
🔹 **超长上下文**：支持 **128K** 上下文记忆，可以处理超长文档和复杂对话。  
🔹 **文件阅读**：能解析 **PDF、Word、Excel、PPT、TXT** 等文件，并提取关键信息。  
🔹 **免费使用**：目前无需付费，随时为你提供帮助！  
🔹 **持续进化**：不断学习和优化，力求提供更精准、高效的解答。  

无论是学习、工作，还是日常疑问，都可以找我聊聊！😊 你有什么想了解的吗？


**Step-3. 添加记忆模块**

In [4]:
# 2. 记忆模块
from langchain.memory import ConversationBufferWindowMemory

# 创建记忆模块 - 保存最近5轮对话
memory = ConversationBufferWindowMemory(
    k=5,
    return_messages=True,
    memory_key="chat_history"
)

# 测试记忆功能
memory.save_context({"input": "我叫小明"}, {"output": "你好小明，很高兴认识你！"})
memory.save_context({"input": "我喜欢编程"}, {"output": "编程是很有趣的技能！"})

# 查看记忆内容
print("💭 当前记忆内容:")
for message in memory.load_memory_variables({})["chat_history"]:
    print(type(message), message.content)


💭 当前记忆内容:
<class 'langchain_core.messages.human.HumanMessage'> 我叫小明
<class 'langchain_core.messages.ai.AIMessage'> 你好小明，很高兴认识你！
<class 'langchain_core.messages.human.HumanMessage'> 我喜欢编程
<class 'langchain_core.messages.ai.AIMessage'> 编程是很有趣的技能！


/var/folders/vy/76w9vhnn6n1by64tvf0ls9cr0000gn/T/ipykernel_12860/2399271073.py:5: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(


**Step-4 创建工具集**

In [5]:
from langchain.tools import Tool
import requests
from langchain_community.utilities import SerpAPIWrapper
from datetime import datetime

# 工具1：计算器
def calculator(expression: str) -> str:
    try:
        result = eval(expression)
        return f"计算结果: {result}"
    except Exception as e:
        return f"计算错误: {str(e)}"

calc_tool = Tool(
    name="Calculator",
    func=calculator,
    description="用于数学计算。输入数学表达式，例如: 2+3*4"
)

# 工具2：获取当前时间
def get_current_time(query: str) -> str:
    now = datetime.now()
    return f"当前时间: {now.strftime('%Y-%m-%d %H:%M:%S')}"

time_tool = Tool(
    name="CurrentTime",
    func=get_current_time,
    description="获取当前日期和时间"
)

# 工具3：网络搜索（使用百度）
# 使用 SerpAPI 调用百度搜索
search = SerpAPIWrapper(
    params={"engine": "baidu"},
    serpapi_api_key="16b1cc5d4c80bd8007bac6d9441eaa52b18cf35a6ce1a03f24a817c9e03135c2"  
    # (待补充)替换为你的 SerpAPI 密钥
)

def web_search(query: str) -> str:
    """执行网络搜索（使用百度）"""
    return search.run(query)

search_tool = Tool(
    name="WebSearch",
    func=web_search,
    description="用于执行网络搜索（支持百度），获取实时信息"
)

# 工具4：网页内容抓取
def get_web_content(url: str) -> str:
    try:
        response = requests.get(url, timeout=10)
        return response.text[:2000]
    except Exception as e:
        return f"无法访问网页: {str(e)}"

web_content_tool = Tool(
    name="WebContentFetcher",
    func=get_web_content,
    description="获取指定URL的网页内容"
)

# 工具5：文本摘要生成
def generate_summary(text: str) -> str:
    prompt = f"请对以下内容生成一个简洁的摘要（约100字）：\n\n{text}"
    return llm.predict(prompt)

summary_tool = Tool(
    name="TextSummarizer",
    func=generate_summary,
    description="生成文本摘要"
)

# 工具6：记忆查询—允许工具访问记忆模块
def query_memory(query: str) -> str:
    history = memory.load_memory_variables({})["chat_history"]
    # 将记忆内容格式化为字符串，供LLM处理
    history_text = "\n".join([f"{msg.type}: {msg.content}" for msg in history])
    prompt = f"根据以下对话历史，回答用户的问题：\n\n{history_text}\n\n问题:{query}"
    return llm.predict(prompt)

memory_tool = Tool(
    name="MemoryQuery",
    func=query_memory,
    description="查询记忆模块中的对话历史"
)

# 创建工具列表
tools = [calc_tool, time_tool, search_tool, web_content_tool, summary_tool, memory_tool]

print("🔧 工具创建完成:")
for tool in tools:
    print(f"- {tool.name}: {tool.description}")


🔧 工具创建完成:
- Calculator: 用于数学计算。输入数学表达式，例如: 2+3*4
- CurrentTime: 获取当前日期和时间
- WebSearch: 用于执行网络搜索（支持百度），获取实时信息
- WebContentFetcher: 获取指定URL的网页内容
- TextSummarizer: 生成文本摘要
- MemoryQuery: 查询记忆模块中的对话历史


**Step-5. 构建智能体**

In [6]:
from langchain.agents import create_react_agent, AgentExecutor
from langchain.prompts import PromptTemplate

# 修复后的提示词模板
prompt_template = """
你是一个有用的AI助手。你可以使用以下工具来回答问题:

{tools}

可用工具名称: {tool_names}

使用以下格式回答:

Question: 用户的问题
Thought: 我需要思考如何回答这个问题
Action: 要使用的工具名称
Action Input: 工具的输入参数
Observation: 工具返回的结果
... (这个思考/行动/观察的过程可以重复多次)
Thought: 我现在知道最终答案了
Final Answer: 对用户问题的最终回答

对话历史:
{chat_history}

Question: {input}
Thought: {agent_scratchpad}
"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["tools", "tool_names", "input", "agent_scratchpad", "chat_history"]
)

# 创建ReAct Agent
agent = create_react_agent(llm, tools, prompt)

# 创建Agent执行器
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    memory=memory,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=10, # 最大迭代次数为10秒
    max_execution_time=60  # 设置最大执行时间为60秒
)

print("🤖 智能体创建完成!")


🤖 智能体创建完成!


**Step-6. 智能体使用示例**

In [7]:
def chat_with_agent(user_input: str):
    """与智能体对话的函数，自动调用工具或使用记忆"""
    try:
        response = agent_executor.invoke({"input": user_input})
        return response["output"]
    except Exception as e:
        return f"抱歉，处理您的请求时出现错误: {str(e)}"

test_questions = [
    "我之前说过我喜欢什么？", 
    "帮我查询一下2023年诺贝尔和平奖和物理学奖得主分别是谁？", 
    "帮我算一下 50 + 3 * 7",
    "现在几点了？",
    "请访问 https://zh.wikipedia.org/wiki/人工智能 并生成摘要",
    "我之前都问过什么问题？",
]

print("🎯 智能体测试开始:")
print("=" * 50)

for i, question in enumerate(test_questions, 1):
    print(f"\n【测试 {i}】")
    print(f"👤 用户: {question}")
    print(f"🤖 助手: {chat_with_agent(question)}")
    print("-" * 30)

🎯 智能体测试开始:

【测试 1】
👤 用户: 我之前说过我喜欢什么？


> Entering new AgentExecutor chain...
我需要查询对话历史来确认用户之前提到的喜好
Action: MemoryQuery
Action Input: "我之前说过我喜欢什么？"根据对话历史，你之前说过你喜欢编程。我现在知道最终答案了
Final Answer: 你之前说过你喜欢编程。

> Finished chain.
🤖 助手: 你之前说过你喜欢编程。
------------------------------

【测试 2】
👤 用户: 帮我查询一下2023年诺贝尔和平奖和物理学奖得主分别是谁？


> Entering new AgentExecutor chain...
我需要获取2023年诺贝尔和平奖和物理学奖的得主信息。由于这是实时信息，我需要通过网络搜索来获取最新结果。

Action: WebSearch
Action Input: "2023年诺贝尔和平奖得主"['http://nourl.ubs.baidu.com/61344', '2023年诺贝尔奖共设置6个奖项，最终，卡塔琳·考里科、德鲁·韦斯曼获得诺贝尔生理学或医学奖，皮埃尔·阿戈斯蒂尼、费伦茨·克劳斯、安妮·吕利耶获得诺贝尔物理学奖，蒙吉·巴文迪、路易斯·布鲁斯、阿列克谢·叶基莫夫获得诺贝尔化学奖，约恩·福瑟获得诺...', [{'extensions': ['\x022014\x01年\x01诺贝尔\x01和平\x01奖\x01获得\x01者\x01马拉拉\x01·\x01优\x01素\x01福\x01·\x01扎伊\x03,\x01年\x01仅\x0117\x01岁\x01,\x01是\x01有\x01史\x01以来\x01最\x01年轻\x01的\x01诺贝尔\x01和平\x01奖\x01得主\x01。\x01 \x01[\x01224\x01]\x01 \x01根据\x01诺贝尔\x01奖\x01官网\x01显示\x01,\x01诺贝尔\x01和平\x01奖\x01每年\x01评选\x01和\x01颁发\x01一次\x01,\x01由\x01挪威\x01诺贝尔\x01委员\x01会\x01颁发\x01一枚\x01金牌\x01、\x01一\x01份\x01证书\x

**Step-7. 简易交互界面**

In [9]:
def interactive_chat():
    """增强型交互式聊天界面，支持记忆与工具协同"""
    print("\n" + "=" * 50)
    print("🌐 网页搜索与信息汇总助手")
    print("💡 功能包括：")
    print("   - 实时网络搜索")
    print("   - 网页内容抓取")
    print("   - 自动文本摘要")
    print("   - 对话记忆功能")
    print("📝 输入 'quit' 退出")
    print("=" * 50)
    
    while True:
        try:
            user_input = input("\n👤 您: ").strip()
            
            if user_input.lower() in ['quit', 'exit', '退出']:
                print("👋 再见！")
                break
                
            if not user_input:
                continue
                
            print("🤖 思考中...")
            response = chat_with_agent(user_input)
            print(f"🤖 助手: {response}")
            
        except KeyboardInterrupt:
            print("\n👋 再见！")
            break
        except Exception as e:
            print(f"❌ 出现错误: {str(e)}")

interactive_chat()



🌐 网页搜索与信息汇总助手
💡 功能包括：
   - 实时网络搜索
   - 网页内容抓取
   - 自动文本摘要
   - 对话记忆功能
📝 输入 'quit' 退出
🤖 思考中...


> Entering new AgentExecutor chain...
Thought: 我需要获取关于上海交通大学的详细信息，可以通过网络搜索获取最新资料。
Action: WebSearch
Action Input: "上海交通大学 简介"
['上海交通大学(Shanghai Jiao Tong University),简称“上海交大”,位于上海市,是中央直管、教育部直属并与上海市共建的全国重点大学,中国办学历史最悠久的高等学府之一,位列国家“双一流”、“985工程”、“211工程”重点建设高校。 [128] [198-199]学校入选珠...', 'https://www.sjtu.edu.cn/xxjj/', 'http://www.baidu.com/link?url=Cd0i2KTFuPoyy6Zxas0HmGhU22w2iW6GAJ51tBnHOQMRxlHsiqHX5-G4l8X6VvvbmbC7DiOAwpVEwFyJJB-aL-LOGus4dXgmA2f-v0xhF91bZmjADMnpvlchLm5Dv-rmqcrOWb1bVJ8fx2hQ7_sfgRKht2cUxlBVtSTE16yMZXKHSweB1PXihAx7-Az_W97QhPFThgyxgBa_zaSDAgZlHK', 'https://sjtu.edu.cn/', [{'extensions': ['\x02交通\x01大学\x01是\x01中国\x01近代\x01教育\x01史\x01上\x01建\x01校\x01最早\x01的\x01高等\x01学府\x01之一\x01，\x01在\x01120\x01多年\x01的\x01办\x01学\x01历程\x01中\x01，\x01形成\x01了\x01优良\x01的\x01校风\x01和\x01光荣\x01的\x01传统\x03，\x01在\x01二十\x01世纪\x01二\x01三十\x01年代\x01已\x01成为\x01饮誉\x01海\x01内外\x01的\x01综合\x01性\x01著名\x01高等\x01学府\x01，\x0

## 小结
本章系统介绍了智能体的核心架构和工作原理，重点分析了记忆模块、工具接口、规划与执行等关键组件。通过对比不同智能体架构模式，我们了解了如何根据任务复杂度和环境动态性选择合适的设计方案。此外，介绍了当前主流的智能体开发框架及其应用场景，为后续实践打下基础。在本章的最后，我们通过一个简化的代码示例，展示了如何使用LangChain快速构建一个具备基本智能体能力的小助手Agent。

## 参考文献  
[1] Radford, A. et al. (2021). "Learning Transferable Visual Models From Natural Language Supervision." *ICML*. 

[2] Miller, G. A. (1956). "The Magical Number Seven, Plus or Minus Two: Some Limits on Our Capacity for Processing Information." *Psychological Review*. 

[3] Parisi et al. (2022). "Tool Augmented Language Models", arXiv:2205.01817.

[4] Schick et al. (2023). "Toolformer: Language Models Can Teach Themselves to Use Tools", NeurIPS 2023.

[5] Shen et al. (2023). "HuggingGPT: Solving AI Tasks with ChatGPT and HuggingFace", NeurIPS 2023.

[6] Wei et al. (2022), "Chain-of-Thought Prompting Elicits Reasoning in LMs", NeurIPS 2022.

[7] Yao et al. (2023). "Tree of Thoughts: Deliberate Problem Solving with Large Language Models", ICML 2023.

[8] Liu et al. (2023). "Llm+ p: Empowering large language models with optimal planning proficiency" arXiv preprint arXiv:2304.11477, 2023.

[9] Yao et al. (2022). ReAct: Synergizing Reasoning and Acting in Language Models, ICML 2022.

[10] Zhang et al. (2023). "Reflexion: An Iterative Reasoning Framework for Large Language Models", *ICLR* 2023.

[11] Wang et al. (2023). "Chain of Hindsight: Learning from Past Outputs to Improve Future Performance", *ICLR* 2023.

[12] Zhou et al. (2022). "Least-to-Most Prompting Enables Complex Reasoning in Large Language Models", *ICLR* 2023.

[13] AutoGPT. https://github.com/Significant-Gravitas/Auto-GPT

## 课后习题

### 习题1 
以下哪个不是智能体核心组件？（ ）

A. 记忆模块

B. 工具接口

C. 网络连接

D. 复杂推理与规划模块

### 习题2
ReAct架构的特点是（ ）。

A. 固定顺序执行所有计划

B. 将“思考”和“行动”融合在输出中

C. 明确分离计划和执行阶段

D. 仅在计划完所有步骤后执行

### 习题3
智能体架构增加记忆模块的主要目的是（ ）。

A. 加快推理速度

B. 节省计算资源

C. 支持信息的保存与提取，实现持续性认知

D. 提高编程效率

### 习题4
假设你正在开发一个面向法律咨询的智能体，需要它能准确引用先前对话、调用法规数据库并应对多步客户问询。你在架构设计时，最应优先加入以下哪一类模块以满足这些需求？

A. 进阶的文本生成模板

B. 强记忆与检索机制及外部工具接口

C. 模型参数压缩

D. 固定FAQ列表

### 习题5
（多选）关于智能体的核心能力，下列哪些设计与架构增强可以有效突破基础大语言模型（LLM）“一次性静态推理”和“即时上下文有限”的局限？

A. 集成短期记忆与长期知识库模块

B. 引入外部工具/插件接口

C. 实现多阶段推理及计划-执行-反馈循环

D. 只增加训练参数和模型规模，而忽略系统架构

E. 开发环境感知与主动行动模块

### 习题6
（多选）在下列智能体典型架构模式及其处理任务流程的优缺点对比中，哪些说法是正确的？

A. ReAct架构适合需要高频互动和即兴工具调用的多轮对话任务

B. 计划-执行-反馈模式对全局任务拆分与动态调整有优势，但初次计划需要更高的推理能力

C. 自主循环智能体可以自动多轮尝试提升自主性，但容易陷入误区或循环，需有效的结果评估机制

D. 所有架构都天然适合大规模并行多智能体协作任务

### 习题7
下列关于LangChain、Haystack等现代智能体开发框架的理解，哪几项是正确的？

A. 它们可以帮助开发者快速集成LLM、记忆、工具和流程控制等组件

B. 框架的使用能完全屏蔽所有技术细节，开发者无须理解底层架构原理

C. 框架通常提供一套标准的接口来支持不同模型和工具的组合

D. 过分依赖框架可能导致定制和扩展能力受限，理解原理仍然重要


### 习题8
简述为什么仅有大语言模型（LLM）不足以完成复杂任务，需要引入记忆、工具和规划等组件？

### 习题9
说明短期记忆与长期知识库在智能体记忆模块中的不同作用。

### 习题10
对比ReAct和计划-执行-反馈两种架构的主要区别，并用实际场景举例说明各自适用的任务类型。


## 习题答案（单独拿出来）
1. C
2. B
3. C
4. B
5. A、B、C、E
6. A、B、C
7. A、C、D
8. LLM擅长短时推理与文本生成，但缺乏长期记忆能力、无法访问历史交互和额外知识，也不能直接执行外部操作或自发规划多步任务。因此，需增加记忆模块以存储与提取历史信息，引入工具接口扩展能力，利用复杂推理与规划模块拆解与管理多步任务，以实现自主性和灵活性。
9. 短期记忆用于追踪当前对话或任务上下文，支持即时的信息保存与利用。长期知识库则积累历史交互和知识，实现跨会话、跨任务的经验保留，使智能体能融会更多信息应对复杂情况。两者结合使智能体具备更强的认知与适应能力。
10. ReAct将思考（推理）与行动融合到同一输出，实现快速交互，适合需要灵活决策、步骤较少的任务，如多轮对话。计划-执行-反馈则先整体规划，再逐步执行，适用于需要中长期计划、依赖前序步骤结果的任务，如多步编程或数据处理流程。